In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextIteratorStreamer
from huggingface_hub import snapshot_download
from datasets import Dataset
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import Trainer, TrainingArguments
from threading import Thread

In [2]:
local_dir = "../model/kanana-nano-2.1b-instruct"

base_model = AutoModelForCausalLM.from_pretrained(local_dir, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(local_dir, padding_side="left")

model = PeftModel.from_pretrained(
    base_model,
    "./outputs/checkpoint-400"
)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 1792, padding_idx=128001)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1792, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1792, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_p

In [3]:
new_template = """{% for message in messages %}
{% if loop.index0 == 0 %}{{ bos_token }}{% endif %}
<|start_header_id|>{{ message['role'] }}<|end_header_id|>

{% if message['role'] == 'assistant' %}
{% generation %}
{{ message['content'] | trim }}
{% endgeneration %}
{% else %}
{{ message['content'] | trim }}
{% endif %}
<|eot_id|>
{% endfor %}

{% if add_generation_prompt %}
<|start_header_id|>assistant<|end_header_id|>
{% endif %}
"""
tokenizer.chat_template = new_template

In [3]:
generation_config = GenerationConfig(
    max_new_tokens=1024,  # 생성할 최대 토큰 수 (작게 조정)
    temperature=0.7,
    top_p=0.95,
    top_k=50,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,  # 끝 토큰 명시
    pad_token_id=tokenizer.pad_token_id,
)

In [4]:
def stream_with_langchain(question):
    """LangChain과 스트리밍을 함께 사용"""
    messages = [
        {"role": "user", "content": question}
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True, skip_prompt=True)
    
    generation_kwargs = {
        "input_ids": input_ids,
        "generation_config": generation_config,
        "streamer": streamer,
    }
    
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    
    full_response = ""
    for text in streamer:
        print(text, end="", flush=True)
        full_response += text
    
    thread.join()
    return full_response

In [6]:
def stream_base_model(question):
    prompt = question  # 이것이 전부입니다
    
    input_ids = tokenizer(
        prompt,
        return_tensors="pt"
    ).to(model.device)

    streamer = TextIteratorStreamer(
        tokenizer,
        skip_special_tokens=True,
        skip_prompt=True
    )

    generation_kwargs = {
        "input_ids": input_ids["input_ids"],
        "generation_config": generation_config,
        "streamer": streamer,
    }

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    full_response = ""
    for text in streamer:
        print(text, end="", flush=True)
        full_response += text

    thread.join()
    return full_response

In [6]:
# kanana-nano-2.1b-base 알파카 탬플릿 kubeflow 60 스텝 학습
question = "오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼."
print(f"Q: {question}\n")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)
print("\n")

Q: 오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼.

A: 네가 시험을 망쳐서 슬프시겠어요. 부모님께 혼나서 더 슬프시겠어요. 어떤 말씀을 하셨나요? 부모님께 위로받고 싶으신가요?
혹시 부모님께서 그런 말을 해주셨나요?
부모님께서 힘내라고 응원해 주셨나요?
부모님께서 응원해 주셨으면 좋겠어요.
부모님께서 응원해 주셔서 힘내시길 바라요. 지금의 슬픔을 부모님께서 위로해 주셨으면 좋겠어요.
오늘 하루도 힘내세요! 힘내세요! 힘내세요! 
항상 응원할게요.  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 힘내세요!  힘내세요! 힘내세요! 

KeyboardInterrupt: 

In [ ]:
# kanana-nano-2.1b-base 알파카 탬플릿 kubeflow 60 스텝 학습
question = "AI 플랫폼을 소개해주세요."
print(f"Q: {question}\n")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)
print("\n")

Q: AI 플랫폼을 소개해주세요.

A: AI 개발을 위한 도구와 리소스를 제공하는 플랫폼



In [ ]:
# kanana-nano-2.1b-instruct 알파카 탬플릿 kubeflow 10 에포크 학습
question = "AI 플랫폼을 소개해주세요."
print(f"Q: {question}\n")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)
print("\n")

Q: AI 플랫폼을 소개해주세요.

A: AI 플랫폼을 소개하겠습니다.

### 카카오클라우드 AI 서비스

카카오클라우드는 사용자 중심의 디지털 혁신을 위해 AI 서비스를 제공합니다. 주요 기능은 다음과 같습니다:

1. **카카오클라우드 AI 서비스(Develop)**:
   - AI 개발을 위한 도구와 리소스를 제공하여, 사용자가 머신러닝 모델을 쉽게 구축하고 테스트할 수 있도록 지원합니다.
   - Jupyter Notebook을 통해 코드 실행 및 시각화를 통해 머신러닝 모델을 구현하고 최적화할 수 있습니다.

2. **카카오클라우드 AI 서비스(Operate)**:
   - AI 모델 운영을 지원하는 도구와 서비스를 제공합니다. 예를 들어, Kubeflow를 통해 머신러닝 워크플로우를 구성하고 실행할 수 있습니다.
   - AI 모델의 배포, 모니터링, 자동화 등을 효율적으로 관리할 수 있습니다.

3. **카카오클라우드 AI 서비스(Explain)**:
   - 모델의 예측을 설명하고 투명성을 제공하는 도구와 서비스를 제공합니다. SHAP (SHapley Value-based Image Attentions)를 통해 모델의 예측 과정을 분석할 수 있습니다.

4. **카카오클라우드 AI 서비스(Train)**:
   - 대규모 데이터셋을 빠르고 효율적으로 처리할 수 있는 GPU 기반의 컴퓨팅 리소스를 제공합니다.
   - 다양한 머신러닝 프레임워크와 라이브러리를 지원하여, 모델을 쉽게 개발하고 실험할 수 있습니다.

카카오클라우드 AI 서비스는 개발자와 기업이 머신러닝 모델을 효율적으로 구축하고 운영할 수 있도록 다양한 리소스와 도구를 제공합니다.



In [ ]:
# kanana-nano-2.1b-instruct 알파카 탬플릿 감정 대화 50 스텝 학습
question = "오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼."
print(f"Q: {question}\n")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)
print("\n")

Q: 오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼.

A: 오늘 시험을 망쳐서 부모님께 혼나셨군요. 너무 속상하고 슬프셨겠어요. 부모님께 혼나서 슬픈 마음이 풀릴 수 있을까요?



In [ ]:
# kanana-nano-2.1b-instruct 챗 탬플릿 감정 대화 50 스텝 학습
question = "오늘 저녁 메뉴 추천해줘."
print(f"Q: {question}\n")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)
print("\n")

Q: 오늘 저녁 메뉴 추천해줘.

A: 오늘 저녁 메뉴로 추천드리는 것은 "돼지갈비찜"입니다.




In [ ]:
# kanana-nano-2.1b-instruct 챗 탬플릿 assistant only 감정 대화 50 스텝 학습
question = "오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼."
print(f"Q: {question}\n")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)
print("\n")

Q: 오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼.

A: 부모님께 혼나서 기분이 좋지 않으신 것 같아요.
무슨 일이 있었나요?
부모님께 혼난 이유를 말씀해주실 수 있나요?
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서 기분이 나아지시길 바랄게요.
부모님과의 대화에서

In [9]:
# kanana-nano-2.1b-instruct 챗 탬플릿 assistant only 감정 대화 50 스텝 학습
question = "오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼."
print(f"Q: {question}\n")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)
print("\n")

Q: 오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼.

A: 많이 속상하시겠어요.
오늘 시험이 왜 그런 일이 생겼나요?
그러한 일이 생기면 어떻게 하면 좋을까요?
부모님께 속상한 마음을 말씀드릴 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
어떤 방법으로 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
어떤 방법으로 기분을 풀 수 있을까요?
또한 부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
또한 부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있을까요?
부모님과의 대화를 통해 기분을 풀 수 있

In [6]:
# kanana-nano-2.1b-instruct 알파카 탬플릿 감정 대화 50 스텝 학습
question = "오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼."
print(f"Q: {question}\n")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)
print("\n")

Q: 오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼.

A: 어머, 시험을 망쳐서 많이 슬프겠네요. 그래도 부모님께 혼나서 많이 혼나셨겠어요. 기분이 나아지도록 어떻게 하면 좋을까요? 같이 이야기 나눠봐요. 그래도 부모님께서 기운 내라고 응원해 주실 거예요. 힘내세요!



In [7]:
# kanana-nano-2.1b-instruct 알파카 탬플릿 감정 대화 50 스텝 학습
question = "오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼."
print(f"Q: {question}\n")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)
print("\n")

Q: 오늘 시험을 망쳐서 부모님께 혼났어. 너무 슬퍼.

A: 그런 상황이 참 안타깝네요. 시험 결과는 누구에게나 힘들 수 있는 부분이지만, 그로 인해 부모님께 혼나는 것은 정말 마음 아플 것 같아요. 

부모님께서도 너의 감정을 이해하려고 노력하시면 좋을 것 같아요. 혼내시더라도 시험 결과에 대한 구체적인 피드백을 주시고, 앞으로 어떻게 개선할 수 있을지 함께 고민해보면 좋겠어요. 

또한, 스스로를 너무 비난하지 말고, 다음에는 더 잘할 수 있을 거라는 자신감을 가지는 것도 중요해요. 힘든 시간이겠지만, 주변 사람들의 지지와 격려가 큰 힘이 될 거예요.

혹시 친구나 선생님과 이야기하고 싶다면, 그들도 너를 이해하고 도와줄 수 있을 거예요. 슬픈 감정을 털어놓고 함께 해결책을 찾아보는 것도 좋은 방법이에요.

기분이 나아질 수 있는 방법을 찾아보거나, 좋아하는 활동을 하면서 기분을 전환하는 것도 도움이 될 수 있어요. 

언제든지 이야기할 수 있는 사람이 있다는 것을 기억하고, 힘든 시간을 잘 이겨내길 바랍니다.

